In [ ]:
import pandas as pd

data = pd.read_csv("/mnt/c/Users/POOYA/Desktop/data.csv")

data

In [ ]:

def convertAB(Year_AB_1, Year_AB_2, Month_AB, Day_AB, Houre, Minute, Second):
    if Month_AB <=6 and Month_AB >=1:
        if Year_AB_2 >= 20:
            Year = Year_AB_2 + 1300
        else:
            Year = Year_AB_2 + 1400

        Month = Month_AB
        Day = Day_AB
    elif Month_AB <=12 and Month_AB >=7:
        Year = Year_AB_1
        Month = Month_AB
        Day = Day_AB
    else:
        Year = None
        Month = None
        Day = None
    return [int(Year), int(Month), int(Day), int(Houre), int(Minute), int(Second)]



b6 = data[['stationCode', 'Year_AB_1', 'Year_AB_2', 'Month_AB', 'Day_AB', 'BARAN_6', 'BARF_6', 'AB_BARF_6']]
b6['Houre'] = 6
b6['Minute'] = 30
b6['Second'] = 0
b6['JAM_BARAN'] = b6['BARAN_6'] + b6['AB_BARF_6']

b6[['Date']] = b6.apply(
    lambda row : convertAB(row["Year_AB_1"], row["Year_AB_2"], row["Month_AB"], row["Day_AB"], row["Houre"], row["Minute"], row["Second"]),
     axis=1
    )

b6 = b6[['stationCode', 'Date', 'BARAN_6', 'BARF_6', 'AB_BARF_6', 'JAM_BARAN']]
b6.columns = ['stationCode', 'DATE', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']



b18 = data[['stationCode', 'Year_AB_1', 'Year_AB_2', 'Month_AB', 'Day_AB', 'BARAN_18', 'BARF_18', 'AB_BARF_18']]
b18['Houre'] = 18
b18['Minute'] = 30
b18['Second'] = 0
b18['JAM_BARAN'] = b18['BARAN_18'] + b18['AB_BARF_18']

b18[['Date']] = b18.apply(
    lambda row : convertAB(row["Year_AB_1"], row["Year_AB_2"], row["Month_AB"], row["Day_AB"], row["Houre"], row["Minute"], row["Second"]),
     axis=1
    )

b18 = b18[['stationCode', 'Date', 'BARAN_18', 'BARF_18', 'AB_BARF_18', 'JAM_BARAN']]
b18.columns = ['stationCode', 'DATE', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']




data = pd.concat([b6, b18])
data[['YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND']] = pd.DataFrame(data.DATE.tolist(), index= data.index)
data['DATE'] = data['YEAR'].astype(str) + "-" + data['MONTH'].astype(str).str.zfill(2) + "-" + data['DAY'].astype(str).str.zfill(2) + " " + data['HOURE'].astype(str).str.zfill(2) + ":" + data['MINUTE'].astype(str).str.zfill(2) + ":" + data['SECOND'].astype(str).str.zfill(2) 





data = data[['stationCode', 'DATE', 'YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']]


data.sort_values(by=['stationCode', 'YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND'], inplace=True)



In [ ]:
data

In [ ]:
data.to_csv("data.csv", index=False)

In [ ]:
import pandas as pd
import sqlite3
db_path_precipitation = 'App/dashApp/precipitation/precipitation.sqlite'
db_precipitation = sqlite3.connect(db_path_precipitation, check_same_thread=False)





date = "1398/3/1"
time = "18:30:00"
data = [
    {'stationName': 'فریمان', 'baran': 0, 'barf': 0, 'ab_barf': 0}, 
    {'stationName': 'امور آب سبزوار', 'baran': 0, 'barf': 0, 'ab_barf': 0}, 
    {'stationName': 'چناران', 'baran': 0, 'barf': 0, 'ab_barf': 0},
]

In [ ]:
        exist_precipitation_data = pd.read_sql_query(sql="SELECT * FROM precipitation", con=db_precipitation)
        exist_precipitation_data_columns = list(exist_precipitation_data.columns)
        exist_precipitation_data['DATE'] = exist_precipitation_data['YEAR'].astype(str) + "-" + exist_precipitation_data['MONTH'].astype(str).str.zfill(2) + "-" + exist_precipitation_data['DAY'].astype(str).str.zfill(2) + " " + exist_precipitation_data['HOURE'].astype(str).str.zfill(2) + ":" + exist_precipitation_data['MINUTE'].astype(str).str.zfill(2) + ":" + exist_precipitation_data['SECOND'].astype(str).str.zfill(2)
        exist_precipitation_data["uniqueCode"] = exist_precipitation_data["stationCode"].astype(str) + "-" + exist_precipitation_data["DATE"]

In [ ]:
exist_station = pd.read_sql_query(sql="SELECT * FROM station", con=db_precipitation)

In [ ]:
exist_station

In [ ]:
df = pd.DataFrame(columns=exist_precipitation_data_columns[1:] + ["stationName"])

date = date.split("/")
time = time.split(":")

for i in range(len(data)):
    df.loc[i,"stationCode"] = exist_station[exist_station['stationName'] == data[i]["stationName"]].iloc[0,1]
    df.loc[i,"YEAR"] = date[0]
    df.loc[i,"MONTH"] = date[1].zfill(2)
    df.loc[i,"DAY"] = date[2].zfill(2)
    df.loc[i,"HOURE"] = time[0].zfill(2)
    df.loc[i,"MINUTE"] = time[1].zfill(2)
    df.loc[i,"SECOND"] = time[2].zfill(2)
    df.loc[i,"BARAN"] = data[i]["baran"]
    df.loc[i,"BARF"] = data[i]["barf"]
    df.loc[i,"AB_BARF"] = data[i]["ab_barf"]
    df.loc[i,"JAM_BARAN"] = data[i]["baran"] + data[i]["ab_barf"]
    df.loc[i,"stationName"] = data[i]["stationName"]

df['DATE'] = df['YEAR'].astype(str) + "-" + df['MONTH'].astype(str) + "-" + df['DAY'].astype(str) + " " + df['HOURE'].astype(str) + ":" + df['MINUTE'].astype(str) + ":" + df['SECOND'].astype(str)

df["uniqueCode"] = df["stationCode"].astype(str) + "-" + df["DATE"]


In [ ]:
df

In [ ]:
df.loc[df.duplicated(subset=["stationName"]),:]["stationName"].unique().tolist()

In [ ]:
df["uniqueCode"].isin(exist_precipitation_data["uniqueCode"]).any()

In [ ]:
df.loc[df["uniqueCode"].isin(exist_precipitation_data["uniqueCode"]), :]

In [ ]:
from collections import Counter
a = list((Counter(df["uniqueCode"]) - Counter(exist_precipitation_data["uniqueCode"])).elements())
a

In [ ]:
list(set(list(df["uniqueCode"])) - set(a))

In [ ]:
a

In [ ]:
list(df["uniqueCode"])

In [ ]:
df

In [ ]:
df.drop(['uniqueCode', 'DATE', 'stationName'], axis=1)

In [ ]:
exist_precipitation_data

DAILY DATA

In [1]:
import pandas as pd

data = pd.read_csv("data1.csv")

persian_month = {
    'فروردین': 1,
    'اردیبهشت': 2,
    'خرداد': 3,
    'تیر': 4,
    'مرداد': 5,
    'شهریور': 6,
    'مهر': 7,
    'آبان': 8,
    'آذر': 9,
    'دی': 10,
    'بهمن': 11,
    'اسفند': 12,
}

data["MONTH"] = data["MONTH"].map(persian_month)

data

,YEAR,MONTH,DAY,HOURE,MINUTE,SECOND,110162006,110162090,110165013,110168010,...,110164018,110161006,110111003,110147050,110147060,110147126,110147128,110147082,110147120,110155016
0,1369,7,1,18,30,0,0.0,0.0,0.0,0.0,...,0.536667,0.0,0.0,0.0,0.084101,0.0,0.0,0.0,0.0,0.037085
1,1369,7,2,18,30,0,0.0,0.0,0.0,0.0,...,0.536667,0.0,0.0,0.0,0.084101,0.0,0.0,0.0,0.0,0.037085
2,1369,7,3,18,30,0,0.0,0.0,0.0,0.0,...,0.536667,0.0,0.0,0.0,0.084101,0.0,0.0,0.0,0.0,0.037085
3,1369,7,4,18,30,0,0.0,0.0,0.0,0.0,...,0.536667,0.0,0.0,0.0,0.084101,0.0,0.0,0.0,0.0,0.037085
4,1369,7,5,18,30,0,0.0,0.0,0.0,0.0,...,0.536667,0.0,0.0,0.0,0.084101,0.0,0.0,0.0,0.0,0.037085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11176,1400,2,17,18,30,0,0.0,0.0,0.0,7.5,...,2.800000,0.0,5.5,1.5,2.100000,9.5,5.5,0.0,0.0,0.000000
11177,1400,2,18,18,30,0,0.0,0.0,0.0,4.0,...,0.000000,0.0,7.5,0.0,1.100000,1.0,1.5,0.5,0.0,0.000000
11178,1400,2,19,18,30,0,0.0,0.0,0.0,1.0,...,0.500000,0.0,2.0,0.0,0.000000,2.5,0.0,0.5,0.0,0.000000
11179,1400,2,20,18,30,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.5,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [2]:
data = pd.melt(data, id_vars=["YEAR", "MONTH", "DAY", "HOURE", "MINUTE", "SECOND"], var_name="stationCode", value_name="JAM_BARAN")
data['stationCode'] = data['stationCode'].astype(int)
data["BARAN"] = data["JAM_BARAN"]
data["BARF"] = 0.0
data["AB_BARF"] = 0.0
data = data[["stationCode", "YEAR", "MONTH", "DAY", "HOURE", "MINUTE", "SECOND", "BARAN", "BARF", "AB_BARF", "JAM_BARAN"]]
data

data.to_csv('data.csv', index=False)